In [12]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi i'm taegeon, I live in South Korea", "Wow that is so cool!")

In [16]:
memory.load_memory_variables({"input": "what does taegeon like"})

{'history': [SystemMessage(content='On taegeon: taegeon is a person. taegeon lives in South Korea.')]}